<a href="https://colab.research.google.com/github/Alina9/HSE_RecSys2020/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightfm

In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import warnings
from lightfm import LightFM
from lightfm.evaluation import auc_score

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
warnings.filterwarnings('ignore') 

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

In [5]:
from tqdm.notebook import tqdm

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
path = '/content/gdrive/My Drive/Colab Notebooks/'

## Датасет MovieLens

In [8]:
ratings = pd.read_csv(path + 'RecSysHSE/ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [9]:
movie_info = pd.read_csv(path + 'RecSysHSE/ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

In [10]:
movie_info = movie_info.sort_values(by = ['movie_id'])
movie_info.head(3)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [11]:
ratings = ratings.sort_values(by = ['user_id', 'movie_id'])

In [12]:
ratings.head(3)

,user_id,movie_id,rating
40,1,1,5
25,1,48,5
39,1,150,5


## implicit dataset

In [13]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]
implicit_ratings['rating'] = 1

Нумерация фильмов и юзеров идет не подряд. Перенумеруем.

In [14]:
users = np.unique(implicit_ratings["user_id"])
user_old2new = {i: j+1 for (j, i) in enumerate(users)}
user_num = len(users)
movies = np.unique(movie_info["movie_id"])
movies_num = len(movies)
old2new = {i: j+1 for (j, i) in enumerate(movies)}
new_id = [i+1 for i, j in enumerate(movies)]

In [15]:
movie_info['movie_id'] = new_id

In [16]:
implicit_ratings['movie_id'] = implicit_ratings['movie_id'].apply(lambda x: old2new[x]) 
implicit_ratings['user_id'] = implicit_ratings['user_id'].apply(lambda x: user_old2new[x])

сделаем предподсчет просмотренных и непросмотренных фильмов для каждого юзера. и разделим данные на train и test

In [17]:
neg_dict = {}
pos_dict = {}
test = np.empty((0,2), int)
train = np.empty((0,2), int)

In [18]:
for i in tqdm(range(user_num)):
    watched_movies_data = implicit_ratings[implicit_ratings['user_id'] == i+1]
    watched_movies = set(watched_movies_data['movie_id'])
    watched_movies_data = np.array(watched_movies_data)[:,:2]    
    unwatched_movies = set(np.arange(1, movies_num + 1)) - watched_movies
    unwatched_movies = np.array(list(unwatched_movies), dtype=int)
    neg_dict[i+1] = unwatched_movies
    pos_dict[i+1] = np.array(list(watched_movies), dtype=int)
    if len(watched_movies) > 1:        
        k = np.random.randint(0, len(watched_movies_data))
        test = np.vstack((test, watched_movies_data[k]))
        train = np.vstack((train, watched_movies_data[:k], watched_movies_data[k+1:]))
        

## Вспомогательные функции

In [19]:
def similar_items(item_id, model, N=10):
    (item_biased, item_representations) = model.get_item_representations()
    item_norms = np.linalg.norm(item_representations, axis=1).reshape(-1,1)
    item_norms[item_norms == 0] = 1e-10
    item_representations /= item_norms 
    scores = item_representations.dot(item_representations[item_id])    
    best = np.argpartition(scores, -N)[-N:]
    return sorted(zip(best, scores[best]), key=lambda x: x[1], reverse=True)

In [20]:
def recommendation(user_id, model, N=10):
    watched_movies = pos_dict[user_id]
    unwatched_movies = neg_dict[user_id]
    scores = model.predict(user_id, unwatched_movies)
    best = sorted(zip(unwatched_movies, scores), key=lambda x: x[1], reverse=True)[:N]
    return best

In [21]:
get_similars = lambda item_id, model : [(movie_info[movie_info["movie_id"] == x[0]]["name"].to_string(), x[1])
                                        for x in similar_items(item_id, model)]

In [22]:
get_recommendations = lambda user_id,  model: [(movie_info[movie_info["movie_id"] == x[0]]["name"].to_string(), x[1]) 
                                               for x in recommendation(user_id, model)]

In [23]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

## Метрики 

In [24]:
def get_hitrate_ndcg(model, test, type_model = "warp"):
    y_true = np.zeros(100, dtype=int)
    y_true[0] = 1 
    ndcg = []
    hitrate = []  
    for i in tqdm(range(len(test))):
        pos_ex = test[i].copy()
        u = pos_ex[0]
        neg_ex = np.vstack((u * np.ones(99), np.random.choice(neg_dict[u], 99, replace=False))).T
        test_u = np.vstack((pos_ex, neg_ex))
        if type_model == "warp":
          y_pred = model.predict(test_u[:,0], test_u[:,1])
        else:
          y_pred = model.predict(test_u)
        ndcg.append(ndcg_score([y_true], 
                               [y_pred], 
                               k=10))
        hitrate.append((np.argsort(y_pred)[::-1][:10] == 0).sum())
    print(f"nDCG@k = {np.mean(ndcg):4f}")
    print(f"hit rate@k = {np.mean(hitrate):4f}")
    return np.mean(ndcg), np.mean(hitrate)

## Sparse matrix

In [25]:
users = train[:,0]
movies = train[:,1]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)), 
                          shape=(user_num+1, movies_num+1))
train_csr = user_item.tocsr()

In [26]:
users = test[:,0]
movies = test[:,1]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)),
                          shape=(user_num+1, movies_num+1))
test_csr = user_item.tocsr()

## WARP

In [27]:
warp = LightFM(no_components = 100,
                loss='warp', 
                learning_schedule='adagrad',
                learning_rate = 0.05,  
                item_alpha = 1e-3,
                user_alpha = 1e-3, 
                max_sampled = 100)


In [28]:
warp.fit(train_csr, epochs=50, verbose=True, num_threads=4)

Epoch: 100%|██████████| 50/50 [02:26<00:00,  2.93s/it]


In [29]:
get_similars(1, warp)

[('0    Toy Story (1995)', 0.9999999),
 ('3045    Toy Story 2 (1999)', 0.9835939),
 ("2286    Bug's Life, A (1998)", 0.96215945),
 ('1245    Groundhog Day (1993)', 0.9582454),
 ('1179    Princess Bride, The (1987)', 0.95182085),
 ('2647    Ghostbusters (1984)', 0.94361424),
 ('2252    Pleasantville (1998)', 0.9310724),
 ('584    Aladdin (1992)', 0.9308308),
 ('2918    Who Framed Roger Rabbit? (1988)', 0.93006325),
 ('1250    Back to the Future (1985)', 0.92690927)]

In [30]:
get_recommendations(4, warp)

[('1178    Star Wars: Episode V - The Empire Strikes Back...', 0.8165656),
 ('1192    Star Wars: Episode VI - Return of the Jedi (1983)', 0.7388539),
 ('585    Terminator 2: Judgment Day (1991)', 0.73195136),
 ('2789    American Beauty (1999)', 0.72457457),
 ('2502    Matrix, The (1999)', 0.7150441),
 ('108    Braveheart (1995)', 0.6964187),
 ('847    Godfather, The (1972)', 0.68702996),
 ('589    Silence of the Lambs, The (1991)', 0.68054193),
 ('1250    Back to the Future (1985)', 0.6618697),
 ('1179    Princess Bride, The (1987)', 0.6484508)]

In [31]:
get_user_history(4, implicit_ratings)

['257    Star Wars: Episode IV - A New Hope (1977)',
 '476    Jurassic Park (1993)',
 '1023    Die Hard (1988)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '1180    Raiders of the Lost Ark (1981)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '1196    Alien (1979)',
 '1220    Terminator, The (1984)',
 '1366    Jaws (1975)',
 '1885    Rocky (1976)',
 '1959    Saving Private Ryan (1998)',
 '2297    King Kong (1933)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '2882    Fistful of Dollars, A (1964)',
 '3349    Thelma & Louise (1991)',
 '3399    Hustler, The (1961)',
 '3633    Mad Max (1979)']

In [32]:
ndcg, hr = get_hitrate_ndcg(warp, test)


nDCG@k = 0.483991
hit rate@k = 0.747391


# NCF

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [81]:
class GMF(nn.Module):
    def __init__(self, embedding_size, num_users, num_items, pos_dict = None, neg_dict = None):
        super(GMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        self.pos_dict = pos_dict
        self.neg_dict = neg_dict

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_size)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_size)

        self.linear = nn.Linear(in_features=self.embedding_size, out_features=1)
        self.sigmoid = nn.Sigmoid()
        self.bceloss = nn.BCELoss()

    def forward(self, user_id, item_id):
        user_embedding = self.embedding_user(user_id)
        item_embedding = self.embedding_item(item_id)
        element_product = torch.mul(user_embedding, item_embedding)
        score = self.sigmoid(self.linear(element_product))
        return score

    def part_forward(self, user_id, item_id):
        user_embedding = self.embedding_user(user_id)
        item_embedding = self.embedding_item(item_id)
        element_product = torch.mul(user_embedding, item_embedding)
        return element_product
    
    def loss(self, user_id, item_id, target):
        y = self.forward(user_id, item_id)
        return self.bceloss(y, target.reshape(-1, 1).float())

    def predict(self, test):
        y_pred = None    
        loader = DataLoader(test, batch_size=512)
        for x in loader:
            users = x[:,0].to(device).long()
            items = x[:,1].to(device).long()
            out = self(users, items)
            if y_pred is None:
                y_pred = out.squeeze().cpu().detach().numpy()
            else:
                y_pred = np.hstack((y_pred, out.squeeze().cpu().detach().numpy()))
        return y_pred

    def embeddings(self):
        self.item_embed = self.embedding_item.weight.detach().cpu().numpy()
        self.user_embed = self.embedding_user.weight.detach().cpu().numpy()
        item_norms = np.linalg.norm(self.item_embed, axis=-1).reshape(-1, 1)
        self.item_embed/=item_norms
        user_norms = np.linalg.norm(self.user_embed, axis=-1).reshape(-1, 1)
        self.user_embed/=user_norms

    def similar_items(self, item_id, N=10):
        scores = self.item_embed @ self.item_embed[item_id]
        best = np.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: x[1], reverse=True)

    def recommendation(self, user_id, N=10):
        unwatched_movies = self.neg_dict[user_id]
        user = np.array([user_id]*len(unwatched_movies))
        data = np.concatenate([user.reshape(-1,1),unwatched_movies.reshape(-1,1)], 1)
        scores = self.predict(data)
        best = sorted(zip(unwatched_movies, scores), key=lambda x: x[1], reverse=True)[:N]
        return best

In [82]:
class MLP(nn.Module):
    def __init__(self, embedding_size, num_users, num_items, layers_size, pos_dict = None, neg_dict = None):
        super(MLP, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        self.layers_size = layers_size
        self.pos_dict = pos_dict
        self.neg_dict = neg_dict

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_size)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_size)
        
        self.layers_size = [2*self.embedding_size] + self.layers_size
        self.linear_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layers_size[:-1], self.layers_size[1:])):
            self.linear_layers.append(nn.Linear(in_size, out_size))

        self.linear = nn.Linear(in_features=self.layers_size[-1], out_features=1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.bceloss = nn.BCELoss()
        
    def forward(self, user_id, item_id):
        user_embedding = self.embedding_user(user_id)
        item_embedding = self.embedding_item(item_id)
        output = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for linear in self.linear_layers:
            output = self.relu(linear(output))
            
        logits = self.linear(output)
        score = self.sigmoid(logits)
        return score
    
    def part_forward(self, user_id, item_id):
        user_embedding = self.embedding_user(user_id)
        item_embedding = self.embedding_item(item_id)
        output = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for linear in self.linear_layers:
            output = self.relu(linear(output))
            
        return output

    def loss(self, user_id, item_id, target):
        y = self.forward(user_id, item_id)
        return self.bceloss(y, target.reshape(-1, 1).float())
    
    def predict(self, test):
        y_pred = None    
        loader = DataLoader(test, batch_size=512)
        for x in loader:
            users = x[:,0].to(device).long()
            items = x[:,1].to(device).long()
            out = self(users, items)
            if y_pred is None:
                y_pred = out.squeeze().cpu().detach().numpy()
            else:
                y_pred = np.hstack((y_pred, out.squeeze().cpu().detach().numpy()))
        return y_pred
    
    def embeddings(self):
        self.item_embed = self.embedding_item.weight.detach().cpu().numpy()
        self.user_embed = self.embedding_user.weight.detach().cpu().numpy()
        item_norms = np.linalg.norm(self.item_embed, axis=-1).reshape(-1, 1)
        self.item_embed/=item_norms
        user_norms = np.linalg.norm(self.user_embed, axis=-1).reshape(-1, 1)
        self.user_embed/=user_norms
    
    def similar_items(self, item_id, N=10):
        scores = self.item_embed @ self.item_embed[item_id]
        best = np.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: x[1], reverse=True)

    def recommendation(self, user_id, N=10):
        unwatched_movies = self.neg_dict[user_id]
        user = np.array([user_id]*len(unwatched_movies))
        data = np.concatenate([user.reshape(-1,1),unwatched_movies.reshape(-1,1)], 1)
        scores = self.predict(data)
        best = sorted(zip(unwatched_movies, scores), key=lambda x: x[1], reverse=True)[:N]
        return best

In [76]:
class NCF(nn.Module):
    def __init__(self, embedding_size, num_users, num_items, layers_size, pos_dict, neg_dict):
        super(NCF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        self.layers_size = layers_size
        self.pos_dict = pos_dict
        self.neg_dict = neg_dict
        
        self.MLP = MLP(embedding_size, num_users, num_items, layers_size)
        self.GMF = GMF(embedding_size, num_users, num_items)
        
        self.linear = nn.Linear(layers_size[-1] + embedding_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.bceloss = nn.BCELoss()
        
    def forward(self, user_id, item_id):
        mlp_output = self.MLP.part_forward(user_id, item_id)
        gmf_output = self.GMF.part_forward(user_id, item_id)
        output = torch.cat([mlp_output, gmf_output], dim=-1)
        score = self.sigmoid(self.linear(output))        
        return score
    
    def loss(self, user_id, item_id, target):
        y = self.forward(user_id, item_id)
        return self.bceloss(y, target.reshape(-1, 1).float())

    def predict(self, test):
        y_pred = None    
        loader = DataLoader(test, batch_size=512)
        for x in loader:
            users = x[:,0].to(device).long()
            items = x[:,1].to(device).long()
            out = self(users, items)
            if y_pred is None:
                y_pred = out.squeeze().cpu().detach().numpy()
            else:
                y_pred = np.hstack((y_pred, out.squeeze().cpu().detach().numpy()))
        return y_pred

    def embeddings(self):
        self.item_embed = np.hstack((self.GMF.embedding_item.weight.detach().cpu().numpy(), 
                            self.MLP.embedding_item.weight.detach().cpu().numpy()))
        self.user_embed = np.hstack((self.GMF.embedding_user.weight.detach().cpu().numpy(), 
                            self.MLP.embedding_user.weight.detach().cpu().numpy()))
        item_norms = np.linalg.norm(self.item_embed, axis=-1).reshape(-1, 1)
        self.item_embed/=item_norms
        user_norms = np.linalg.norm(self.user_embed, axis=-1).reshape(-1, 1)
        self.user_embed/=user_norms
    
    def similar_items(self, item_id, N=10):
        scores = self.item_embed @ self.item_embed[item_id]
        best = np.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: x[1], reverse=True)

    def recommendation(self, user_id, N=10):
        unwatched_movies = self.neg_dict[user_id]
        user = np.array([user_id]*len(unwatched_movies))
        data = np.concatenate([user.reshape(-1,1),unwatched_movies.reshape(-1,1)], 1)
        scores = self.predict(data)
        best = sorted(zip(unwatched_movies, scores), key=lambda x: x[1], reverse=True)[:N]
        return best

In [37]:
class Data:
    def __init__(self, implicit_ratings, unwatched_movies):
        self.implicit_ratings = implicit_ratings
        self.unique_users = np.unique(implicit_ratings[:,0])
        self.unique_items = np.unique(implicit_ratings[:,1])
        self.treshold = len(self.implicit_ratings)
        
        self.watched_movies = {}
        self.unwatched_movies = unwatched_movies

        for u in range(1, len(self.unique_users) + 1):
            self.watched_movies[u] = np.array(implicit_ratings[implicit_ratings[:,0] == u][:,1], dtype=int)
        

    def __getitem__(self, index):
        if index < self.treshold:
            user, item = self.implicit_ratings[index,0], self.implicit_ratings[index,1]
            y=1
        else:
            user = np.random.choice(self.unique_users, 1)[0]
            item = np.random.choice(self.unwatched_movies[user], 1)[0]
            y=0
        return user, item, y

    def __len__(self):
        return 2*self.treshold

In [38]:
def train_model(model, train_loader, optimizer, epoch):
    train_losses = []
    for e in tqdm(range(epoch)):
        model.train()    
        epoch_losses = []
        for user_id, item_id, target in train_loader:
            user_id = user_id.to(device)
            item_id = item_id.to(device)
            target = target.to(device)
            loss = model.loss(user_id, item_id, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())
        if e%5 == 0:
            print(f'Epoch {e}, Train loss {np.mean(epoch_losses)}')
        train_losses.append(np.mean(epoch_losses))

    return train_losses

In [39]:
get_similars = lambda item_id, model: [(movie_info[movie_info["movie_id"] == x[0]]["name"].to_string(), x[1])
                                        for x in model.similar_items(item_id)]

get_recommendations = lambda user_id, model: [(movie_info[movie_info["movie_id"] == x[0]]["name"].to_string(), x[1]) 
                                        for x in model.recommendation(user_id)]

In [40]:
train_data = Data(train, neg_dict)
train_loader = DataLoader(train_data, batch_size=1024, shuffle=True)

In [41]:
num_items = np.max(implicit_ratings["movie_id"])+1
num_users = np.max(implicit_ratings['user_id'])+1

In [83]:
mlp = MLP(embedding_size = 100,num_users = num_users, num_items = num_items, layers_size = [100, 128, 100], pos_dict=pos_dict,
          neg_dict = neg_dict).to(device)
optimizer = optim.Adam(mlp.parameters(), 1e-2)
train_loss = train_model(mlp, train_loader, optimizer, epoch = 50)

Epoch 0, Train loss 0.3902831724811372
Epoch 5, Train loss 0.30957080158505507
Epoch 10, Train loss 0.252731146197096
Epoch 15, Train loss 0.220617319088724
Epoch 20, Train loss 0.20027695037424564
Epoch 25, Train loss 0.18813080332583662
Epoch 30, Train loss 0.17847342409273917
Epoch 35, Train loss 0.1705655303495608
Epoch 40, Train loss 0.16227200980425524
Epoch 45, Train loss 0.1576569453283823



In [84]:
torch.save(mlp.state_dict(), 'mlp.pcl')

In [85]:
mlp.embeddings()
get_similars(1, mlp)

[('0    Toy Story (1995)', 1.0),
 ('1245    Groundhog Day (1993)', 0.78759384),
 ('3045    Toy Story 2 (1999)', 0.7658341),
 ('33    Babe (1995)', 0.7007741),
 ('2252    Pleasantville (1998)', 0.69936085),
 ('584    Aladdin (1992)', 0.6900675),
 ('1595    Full Monty, The (1997)', 0.68354833),
 ('38    Clueless (1995)', 0.67325187),
 ("1854    There's Something About Mary (1998)", 0.6597947),
 ('1120    Monty Python and the Holy Grail (1974)', 0.6588289)]

In [86]:
get_recommendations(4, mlp)

[('453    Fugitive, The (1993)', 0.9888192),
 ('1178    Star Wars: Episode V - The Empire Strikes Back...', 0.9881108),
 ('847    Godfather, The (1972)', 0.9879046),
 ('1203    Godfather: Part II, The (1974)', 0.9860721),
 ('108    Braveheart (1995)', 0.98573905),
 ('1568    Hunt for Red October, The (1990)', 0.98003024),
 ('585    Terminator 2: Judgment Day (1991)', 0.9778397),
 ('912    2001: A Space Odyssey (1968)', 0.9751076),
 ('1192    Star Wars: Episode VI - Return of the Jedi (1983)', 0.97223026),
 ('1182    Aliens (1986)', 0.97135746)]

In [97]:
ndcg, hr = get_hitrate_ndcg(mlp, test, "mlp")


nDCG@k = 0.540103
hit rate@k = 0.794931


In [ ]:
gmf = GMF(embedding_size = 100,num_users = num_users, num_items = num_items,
          pos_dict = pos_dict, neg_dict = neg_dict).to(device)
optimizer = optim.Adam(gmf.parameters(), 1e-2)
train_loss = train_model(gmf, train_loader, optimizer, epoch = 50)

Epoch 0, Train loss 0.6359700821340084
Epoch 5, Train loss 0.22124563263421007
Epoch 10, Train loss 0.14551844630095598
Epoch 15, Train loss 0.12363653299700228
Epoch 20, Train loss 0.11095296659526958
Epoch 25, Train loss 0.10519833636316035
Epoch 30, Train loss 0.09880313544581049
Epoch 35, Train loss 0.09556728786380171
Epoch 40, Train loss 0.0924173431544692


In [106]:
torch.save(gmf.state_dict(), 'gmf.pcl')

In [107]:
gmf.embeddings()
get_similars(1, gmf)

[('0    Toy Story (1995)', 1.0),
 ('3045    Toy Story 2 (1999)', 0.6126512),
 ('33    Babe (1995)', 0.53128445),
 ('360    Lion King, The (1994)', 0.47397986),
 ('584    Aladdin (1992)', 0.45034638),
 ('1132    Wrong Trousers, The (1993)', 0.39335677),
 ('1205    Grand Day Out, A (1992)', 0.38524514),
 ('591    Beauty and the Beast (1991)', 0.37919667),
 ("2286    Bug's Life, A (1998)", 0.35832733),
 ('1838    Mulan (1998)', 0.34965676)]

In [108]:
get_recommendations(4, gmf)

[('847    Godfather, The (1972)', 0.9914949),
 ('3361    Death Wish (1974)', 0.9904488),
 ('1203    Godfather: Part II, The (1974)', 0.97998065),
 ('1178    Star Wars: Episode V - The Empire Strikes Back...', 0.92823887),
 ('900    Casablanca (1942)', 0.88207537),
 ('2219    Thing, The (1982)', 0.8341527),
 ('3634    Mad Max 2 (a.k.a. The Road Warrior) (1981)', 0.79638946),
 ('1284    Butch Cassidy and the Sundance Kid (1969)', 0.7788475),
 ('1267    Ben-Hur (1959)', 0.73371387),
 ('585    Terminator 2: Judgment Day (1991)', 0.66102064)]

In [109]:
ndcg, hr = get_hitrate_ndcg(gmf, test, "gmf")


nDCG@k = 0.538939
hit rate@k = 0.787974


In [113]:
ncf = NCF(embedding_size = 100, num_users = num_users, num_items = num_items, layers_size = [100, 128, 100], pos_dict = pos_dict, neg_dict = neg_dict)
ncf.to(device)
ncf.GMF.load_state_dict(torch.load('gmf.pcl'))
ncf.MLP.load_state_dict(torch.load('mlp.pcl'))
optimizer = optim.Adam(ncf.parameters(), 1e-2)
train_loss = train_model(ncf, train_loader, optimizer, epoch = 10)

Epoch 0, Train loss 0.16196688337640153
Epoch 5, Train loss 0.07829932346538031



In [114]:
ncf.embeddings()

In [115]:
get_similars(1, ncf)

[('0    Toy Story (1995)', 1.0),
 ('3045    Toy Story 2 (1999)', 0.71727395),
 ('1245    Groundhog Day (1993)', 0.6926808),
 ('33    Babe (1995)', 0.63970494),
 ('584    Aladdin (1992)', 0.5929309),
 ('360    Lion King, The (1994)', 0.59218526),
 ('2327    Shakespeare in Love (1998)', 0.58631134),
 ("2286    Bug's Life, A (1998)", 0.57700336),
 ('38    Clueless (1995)', 0.57248425),
 ('1250    Back to the Future (1985)', 0.5661273)]

In [116]:
get_recommendations(4, ncf)

[('1178    Star Wars: Episode V - The Empire Strikes Back...', 0.9947135),
 ('847    Godfather, The (1972)', 0.9883957),
 ('1203    Godfather: Part II, The (1974)', 0.9852673),
 ('1284    Butch Cassidy and the Sundance Kid (1969)', 0.98042285),
 ('2875    Dirty Dozen, The (1967)', 0.95028085),
 ('1267    Ben-Hur (1959)', 0.92395777),
 ('585    Terminator 2: Judgment Day (1991)', 0.88087815),
 ('589    Silence of the Lambs, The (1991)', 0.8741353),
 ('1884    French Connection, The (1971)', 0.85072803),
 ('2219    Thing, The (1982)', 0.8497852)]

In [117]:
ndcg, hr = get_hitrate_ndcg(ncf, test, "ncf")


nDCG@k = 0.561115
hit rate@k = 0.811993


# Attention model (недоделанный)

In [118]:
class ComiRecDataloader:
    def __init__(self, pos_dict, neg_dict):
      self.pos_dict = pos_dict        

    def __getitem__(self, index):
        user, history = index, self.pos_dict[index]
        return user, torch.tensor(history, dtype=torch.long)

    def __len__(self):
        return len(self.pos_dict)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, dim):
        super(AttentionHead, self).__init__()
        self.attention_layers = nn.Sequential(
            nn.Linear(dim, dim),
            nn.Tanh(),
            nn.Linear(dim, 1))
        
    def forward(self, history):
        w = self.attention_layers(history)
        w = F.softmax(w, dim=0)
        return history.T @ w

In [ ]:
class ComiRecModel:
    def __init__(self, embedding_size, num_users, num_items):
        super(ComiRecModel, self).__init__()
        self.item_emb = nn.Embedding(num_items, embedding_size)
        self.attention_head = AttentionHead(embedding_size)
        self.num_users = num_users
        self.num_items = num_items

    def (user_embeddings, item_embeddings):
        return (user_embeddings.T @ item_embeddings).max(axis=1)[0]

    def forward(self, history):
        out = self.attention_head(history)
        